In [1]:
from IPython.display import clear_output
! pip install torchmetrics
!pip install lion-pytorch
clear_output()

In [2]:
import os
import math
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.core.dtypes.cast import maybe_box_datetimelike
from copy import deepcopy
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn.modules.container import Sequential
from torch.utils.data import random_split, TensorDataset, Dataset, DataLoader
from torchmetrics import PearsonCorrCoef, R2Score, MeanSquaredError

import torchvision
from torchvision import datasets


from drive.MyDrive.Model_and_trainer import CustomStructureDataset, First_CNN, ProgressPlotter, BaseTrainer

import random


def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [3]:
#Загрузка train, val, test датасетов
train_dataset = CustomStructureDataset('drive/MyDrive/Dataset/Data_files_10_clear_added_2.csv', str_dir = 'drive/MyDrive/Dataset')
val_dataset = CustomStructureDataset('drive/MyDrive/Dataset/Val_data_files.csv', str_dir = 'drive/MyDrive/Dataset')
test_dataset = CustomStructureDataset('drive/MyDrive/Dataset/Resol_test.csv', str_dir = 'drive/MyDrive/Dataset')

In [4]:
# расчет среднего и дисперсии для нормализации целевых значений
train_labels = torch.Tensor(list(train_dataset.str_labels[2].astype(float)))
mean = torch.mean(train_labels).item()
std = torch.std(train_labels).item()
print(mean, std)

7.477478981018066 2.196446180343628


In [5]:
# расчет весов, в случае обучения с weightedsampler
weights = []
for i in train_labels:
  delta = abs(i-mean).item()
  if delta<=1:
    delta=1
  weights.append(delta)

In [6]:

train_dataset.normalize = True # нормализация целевых значений
train_dataset.mean = mean
train_dataset.std = std
train_dataset.train = True # в случае, если train = True, с вероятностью 0.5 происходит замена -1 на 1 и наоборот
train_dataset.transform = True # в случае, если transform = True, с вероятностью 0.5 происходит зеркальное отражение по одной или нескольким осям


val_dataset.train = False
val_dataset.normalize = True
val_dataset.mean = mean
val_dataset.std = std
val_dataset.transform = None

test_dataset.train = False
test_dataset.normalize = True
test_dataset.mean = mean
test_dataset.std = std
test_dataset.transform = None


In [7]:
from torch.utils.data import WeightedRandomSampler
batch_size = 32
#weightedsampler = WeightedRandomSampler(weights, len(weights))
#one_list = [1 for i in range(len(weights))]
#weightedsampler = WeightedRandomSampler(one_list, len(weights))

#train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler = weightedsampler, num_workers=2)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

val_loader = DataLoader(val_dataset, shuffle=False, batch_size=batch_size, num_workers=2)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size, num_workers=2)

In [8]:
# создание модели
model = First_CNN().double()
model = model.to(device)

In [9]:
trainer = BaseTrainer(
    model= model,
    train_dataloader = train_loader,
    test_dataloader= val_loader)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_random_seed(42)
trainer.num_epochs = 20
trainer.score_function = MeanSquaredError()
trainer.quality_metric = 'RMSE'
trainer.optimizer = torch.optim.AdamW(trainer.model.parameters(), lr=0.0001, weight_decay = 0.001)
trainer.fit()

In [ ]:
#проверка модели на тесте
y_test_pred, y_test_true = trainer.get_predictions(model=trainer.best_model, dl=test_loader)
pearson = PearsonCorrCoef()
corr_coef= pearson(y_test_pred, y_test_true)
mse = MeanSquaredError()
rmse = torch.sqrt(mse(y_test_pred, y_test_true))
print(f'Pearson_corr: {round(corr_coef.item(), 2)}')
print(f'RMSE: {round(rmse.item(), 3)}')

In [ ]:
torch.save(trainer.best_model, 'drive/MyDrive/Model/model_best_55_weight_mol_dyn.pt')

In [ ]:
import pandas as pd
corr_df = pd.DataFrame(columns=['Pred_pKD', 'True_pKD'])
corr_df['Pred_pKD'] = y_test_pred
corr_df['True_pKD'] = y_test_true
corr_df

,Pred_pKD,True_pKD
0,7.23,9.16
1,6.87,8.41
2,7.26,5.00
3,6.47,6.60
4,7.05,7.48
5,8.58,9.62
6,7.77,10.33
7,6.33,5.46
8,6.69,6.37
9,8.31,7.98


In [ ]:
import seaborn as sns
sns.jointplot(data=corr_df, x='True_pKD', y='Pred_pKD', palette='Set2', ylim=(2, 14), xlim=(2, 14))

In [ ]:
# 0: Акцепторы вс 1 белка + доноры 2 белка
# 1: Акцепторы вс 2 белка + доноры 1 белка
# 2: Акцепторы вс 1 белка + слабые доноры 2 белка
# 3: Акцепторы вс 2 белка + слабые доноры 1 белка
# 4: пол. заряженные атомы 1 белка + нег. заряженные атомы 2 белка
# 5: пол. заряженные атомы 2 белка + нег. заряженные атомы 1 белка
# 6: Гидрофобные атомы 1 белка + гидрофобные атомы 2 белка
# 7: Карбоксильные кислороды 1 белка + Карбоксильные кислороды 2 белка
# 8: Карбоксильные углероды 1 белка + Карбоксильные углероды 2 белка
# 9: Ароматические атомы 1 белка + ароматические атомы 2 белка
channels_names = ['HB_Ac1+Don2', 'HB_Ac2+Don1', 'HB_Ac1+Weak_Don2', 'HB_Ac2+Weak_Don1', 'Pos1+Neg2', 'Pos2+Neg1', 'Hph1+Hph2', 'Carboxy_C1+Carboxy_C2', 'Carboxy_O1+Carboxy_O2', 'Arom1+Arom2']

In [ ]:
# вывод весов для каждого из каналов на входе
w0 = trainer.best_model.conv_stack[0].weight
w0 = pd.DataFrame(np.transpose(w0.cpu().detach().numpy(), [0, 4, 2, 3, 1]).reshape((-1, 10)),
                  columns=channels_names)

In [ ]:
# расчет среднего значения для значимых нейронов
diff = (w0.abs() > 0.001).mean()
diff.sort_values(ascending=False)

In [ ]:
# range between 25th and 75th percentiles
perc_diff = ((w0.apply(lambda x: np.percentile(x, 75))
             - w0.apply(lambda x: np.percentile(x, 25)))
             .sort_values(ascending=False))

# построение разбросов весов для каждого из 10 каналов
fig, ax = plt.subplots(figsize=(7, 6))

sns.boxplot(data=w0, fliersize=0, orient='h', order=list(perc_diff.index.values), ax=ax)
sns.boxplot(data=w0, fliersize=0, orient='h', ax=ax)
ax.set_xlim(-0.055, 0.055)
ax.set_xticks(np.arange(-0.04, 0.05, 0.02))
ax.set_ylim(10, -1)

fig.tight_layout()